In [1]:
from pulp import *
import pandas as pd
import os 
path = os.getcwd()

In [2]:
param = pd.read_excel("{0}/assignment_ps.xlsx".format(path))

ipykernel_launcher:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.


In [3]:
param

,Unnamed: 0,demand,fixed cost,var,storage cost,Capacity
0,period 1,100,2000,40,1,5000
1,period 2,4000,2000,50,1,6000
2,period 3,2500,2000,80,1,4000
3,period 4,4000,2000,60,1,12000
4,period 5,6000,2000,50,1,4000
5,period 6,100,2000,60,1,4000
6,period 7,250,3000,20,1,8000
7,period 8,5000,3000,40,1,4000
8,period 9,6000,3000,90,1,4000
9,period 10,4000,3000,30,1,7000


In [4]:
param = param.rename(columns={'Unnamed: 0': 'period'})

In [7]:
param['t'] = range(1,13)

In [8]:
param

,period,demand,fixed cost,var,storage cost,Capacity,t
0,period 1,100,2000,40,1,5000,1
1,period 2,4000,2000,50,1,6000,2
2,period 3,2500,2000,80,1,4000,3
3,period 4,4000,2000,60,1,12000,4
4,period 5,6000,2000,50,1,4000,5
5,period 6,100,2000,60,1,4000,6
6,period 7,250,3000,20,1,8000,7
7,period 8,5000,3000,40,1,4000,8
8,period 9,6000,3000,90,1,4000,9
9,period 10,4000,3000,30,1,7000,10


In [9]:
param = param.set_index('t')

In [10]:
inventory = LpVariable.dicts('inv', [0,1,2,3,4,5,6,7,8,9,10,11,12], 0, None, 'Integer')
inventory[0] = 200

In [11]:
production = LpVariable.dicts('Prod', [1,2,3,4,5,6,7,8,9,10,11,12], 0, None, 'Integer')
binary = LpVariable.dicts('binary', [1,2,3,4,5,6,7,8,9,10,11,12], 0, None, 'Binary')

In [18]:
time = [i+1 for i in range(0,12)]

In [20]:
model = LpProblem('Production', LpMinimize)

In [21]:
model += lpSum([inventory[t] * param.loc[t, 'storage cost'] + production[t] * param.loc[t, 'var'] + binary[t] * param.loc[t, 'fixed cost'] for t in time])

In [22]:
for t in time:
    model += production[t] - inventory[t] + inventory[t-1] >= param.loc[t, 'demand']
    model += production[t] <= binary[t] * param.loc[t, 'Capacity']

In [23]:
model.solve()

1

In [24]:
for i in production:
    print(production[i], production[i].varValue)

Prod_1 5000.0
Prod_2 6000.0
Prod_3 0.0
Prod_4 1400.0
Prod_5 4000.0
Prod_6 1250.0
Prod_7 8000.0
Prod_8 4000.0
Prod_9 0.0
Prod_10 7000.0
Prod_11 4000.0
Prod_12 0.0
